<a href="https://colab.research.google.com/github/MAGNNATTA/telecom_x_etl_eda.ipynb/blob/main/Telecom_x_etl_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

url = "https://raw.githubusercontent.com/sthemonica/alura-voz/refs/heads/main/Dados/Telco-Customer-Churn.json"

response = requests.get(url)
data = response.json()

df = pd.DataFrame(data)

df

In [ ]:
import pandas as pd
df_plano = pd.json_normalize(data)
df_plano

In [ ]:
df_plano.info()

In [ ]:
# Verifique se há valores únicos em cada coluna.
for col in df_plano.columns:
  print(f"valores únicos en la columna'{col}': {df_plano[col].nunique()}")
  if df_plano[col].nunique() < 50: # Imprima os valores apenas se não forem muitos.
    print(df_plano[col].unique())
    print('-' * 50)

In [ ]:
#Duplicados
print("Numero de duplicados", df_plano.duplicated().sum())

In [ ]:
# Nulos
print("Numero de Nulos", df_plano.isnull().sum())

In [ ]:
# Vazios ou em branco
df_plano.apply(lambda x: x.astype(str).str.strip() == '').sum()

In [ ]:
#Alterando a coluna "account.Charges.Total" para Float

df_plano['account.Charges.Total'] = pd.to_numeric(df_plano['account.Charges.Total'], errors='coerce')
print(df_plano['account.Charges.Total'].dtype)

In [ ]:
# Eliminando as colunas de Churn

df_plano = df_plano[df_plano['Churn'].str.strip() != '']
print("Número de filas después de eliminar las vacías en 'Churn':", len(df_plano))

In [ ]:
# Vazios ou em branco
df_plano.apply(lambda x: x.astype(str).str.strip() == '').sum()

In [ ]:
df_plano = df_plano.drop('Total.Day', errors='ignore')

In [ ]:
# Calculando os gastos diários
df_plano['Total.Day'] = ((df_plano['account.Charges.Total'] / df_plano['customer.tenure']) /30 ).round(2)

# Inserindo a coluna na posição 19
df_plano.insert(19, 'Total.Day', df_plano.pop('Total.Day'))

In [ ]:
df_plano

In [ ]:
import plotly.express as px

In [ ]:
px.histogram(df_plano, x = 'customer.gender', text_auto = True, color = 'Churn', barmode = 'group')

In [ ]:
px.histogram(df_plano, x = 'customer.SeniorCitizen', text_auto = True, color = 'Churn', barmode = 'group')

In [ ]:
px.histogram(df_plano, x = 'customer.tenure', text_auto = True, color = 'Churn', barmode = 'group')

In [ ]:
# Agrupar por tenure e Churn
df_grouped = df_plano.groupby(['customer.tenure', 'Churn']).size().unstack(fill_value=0)

# Calcular a % de churn por tenure
df_grouped['churn_rate'] = (df_grouped.get('Yes', 0) / df_grouped.sum(axis=1)) * 100

# Mostrar o resultado
print(df_grouped[['churn_rate']])


In [ ]:
fig = px.line(df_grouped.reset_index(),
              x='customer.tenure',
              y='churn_rate',
              markers=True,
              title='Taxa de Churn (%) por Tenure')
fig.show()

In [ ]:
px.scatter(df_plano,
           x="customer.tenure",
           y="account.Charges.Monthly",
           color="Churn",
           title="Tenure vs Encargos mensais por Churn")

In [ ]:
px.histogram(df_plano, x = 'account.Contract', text_auto = True, color = 'Churn', barmode = 'group')

In [ ]:
px.box(df_plano, x = 'Churn', y = 'account.Charges.Monthly', color = 'Churn')

In [ ]:
px.histogram(df_plano, x = 'phone.PhoneService',  text_auto = True, color = 'Churn', barmode = 'group')

In [ ]:
px.histogram(df_plano, x = 'internet.InternetService',  text_auto = True, color = 'Churn', barmode = 'group')

In [ ]:
px.histogram(df_plano, x = 'internet.StreamingTV',  text_auto = True, color = 'Churn', barmode = 'group')

In [ ]:
px.histogram(df_plano, x = 'account.PaymentMethod',  text_auto = True, color = 'Churn', barmode = 'group')

In [ ]:
"""
Relatório Final – Análise de Churn

Introdução:
A evasão de clientes é...
"""

Relatório Final — Análise de Churn de Clientes
📌 Introdução
A evasão de clientes (churn) é um desafio constante no setor de telecomunicações. Compreender os fatores que levam os clientes a cancelar seus serviços é essencial para criar estratégias eficazes de retenção. O objetivo desta análise é aplicar um processo completo de ETL (Extração, Transformação e Carga) e EDA (Análise Exploratória de Dados) com Python, a fim de identificar padrões relevantes e propor soluções que reduzam o churn.

⚙️ Processo de ETL
🗂 Extração e Importação
Os dados foram carregados em um DataFrame com Pandas. Foram feitas inspeções iniciais com métodos como df.info() e df.describe() para entender a estrutura e qualidade do conjunto de dados.

🧼 Limpeza e Transformações
Tratamento de nulos e brancos: Substituímos espaços em branco por NaN e limpamos os dados vazios da variável alvo Churn.
Conversão de dados: Campos como 'account.Charges.Total' foram convertidos para tipo float com tratamento de erros.
Remoção de ruído: Eliminação de colunas desnecessárias como 'Total.Day' original.
Criação de nova variável: Criamos Total.Day, representando o gasto diário médio do cliente, normalizando o valor por mês e por dia.
Reorganização e integridade: Checamos e removemos duplicatas e reordenamos colunas para melhor leitura.
🔍 Análise Exploratória (EDA)
Utilizamos Plotly Express para gerar gráficos interativos e identificar padrões relevantes.

🎯 Principais Insights Visuais
Tenure: A taxa de churn é mais alta nos primeiros 12 meses e estabiliza com o tempo. Isso foi confirmado com um gráfico de linha de churn por mês de contrato.
Encargos Mensais: Clientes que cancelam tendem a ter mensalidades mais altas, como mostrado em um boxplot.
Tipo de Contrato: Contratos mensais apresentaram os maiores índices de churn, enquanto contratos anuais/bianuais indicaram maior fidelidade.
Serviço de Internet: Clientes com fibra óptica mostraram maior propensão ao churn.
Streaming de TV: Leve aumento no churn entre usuários desse serviço.
Forma de Pagamento: Pagamento automático (ex: débito) está associado a menor churn. Já clientes que utilizam boleto ou fatura online mostraram maior evasão.
✅ Conclusão
Através do processo de ETL e EDA foi possível:

Identificar que o perfil de maior risco de churn envolve clientes com contrato mensal, menor tempo de casa e encargos mensais altos.
Observar que certos serviços e métodos de pagamento influenciam diretamente na permanência do cliente.
Produzir gráficos explicativos que reforçam a correlação entre variáveis e o comportamento de cancelamento.
💡 Recomendações Estratégicas
Foco nos primeiros 12 meses: Criar campanhas de onboarding e retenção para novos clientes.
Incentivar contratos mais longos: Oferecer descontos ou benefícios para migração de planos mensais para anuais.
Monitorar valores altos de cobrança: Clientes com mensalidades elevadas devem ser acompanhados de perto.
Promover pagamento automático: Campanhas para incentivar esse método com vantagens exclusivas.
Análise contínua de serviços extras: Streaming, serviços de internet e telefonia devem ser otimizados com base na satisfação e comportamento de churn.
Ferramentas utilizadas: Python, Pandas, Plotly, Matplotlib, Seaborn
Técnicas aplicadas: Limpeza de dados, normalização, visualização interativa, agrupamentos e análise percentual.